# ECSE 420 - Final Project
Parallelized convolution, matrix inversion, and matrix multiplication algorithms.

In [0]:


 !apt-get install nvidia-cuda-toolkit
!pip3 install numba

import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/lib/x86_64-linux-gnu/libnvvm.so"

In [0]:
import math
import multiprocessing
import random
import sys
import time


def merge(*args):
    # Support explicit left/right args, as well as a two-item tuple which works more cleanly with multiprocessing.
    left, right = args[0] if len(args) == 1 else args
    left_length, right_length = len(left), len(right)
    left_index, right_index = 0, 0
    merged = []
    while left_index < left_length and right_index < right_length:
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1
    if left_index == left_length:
        merged.extend(right[right_index:])
    else:
        merged.extend(left[left_index:])
    return merged


def merge_sort(data):
    length = len(data)
    if length <= 1:
        return data
    middle = length // 2
    left = merge_sort(data[:middle])
    right = merge_sort(data[middle:])
    return merge(left, right)


def merge_sort_parallel(data):
    # Creates a pool of worker processes, one per CPU core.
    # We then split the initial data into partitions, sized
    # equally per worker, and perform a regular merge sort
    # across each partition.
    processes = multiprocessing.cpu_count()
    print("parallel processes: {}".format(processes))
    pool = multiprocessing.Pool(processes=processes)
    size = int(math.ceil(float(len(data)) / processes))
    data = [data[i * size:(i + 1) * size] for i in range(processes)]
    data = pool.map(merge_sort, data)
    # Each partition is now sorted - we now just merge pairs of these
    # together using the worker pool, until the partitions are reduced
    # down to a single sorted result.
    while len(data) > 1:
        # If the number of partitions remaining is odd, we pop off the
        # last one and append it back after one iteration of this loop,
        # since we're only interested in pairs of partitions to merge.
        extra = data.pop() if len(data) % 2 == 1 else None
        data = [(data[i], data[i + 1]) for i in range(0, len(data), 2)]
        data = pool.map(merge, data) + ([extra] if extra else [])
    return data[0]


if __name__ == "__main__":
    results = []
    size = int(sys.argv[-1]) if sys.argv[-1].isdigit() else 1000000
    data_unsorted = [random.randint(0, size) for _ in range(size)]
    for sort in merge_sort, merge_sort_parallel:
        start = time.time()
        data_sorted = sort(data_unsorted)
        end = time.time() - start
        results.append(end)
        print(sort.__name__, end, sorted(data_unsorted) == data_sorted)
    print("speedup = {}".format(results[0] / results[1]))

merge_sort 5.2758166790008545 True
parallel processes: 4
merge_sort_parallel 3.1738831996917725 True
speedup = 1.6622592411444783


In [0]:
from numba import cuda
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import random

In [0]:
@cuda.jit
# perform full MergeSort on supplied subsection of data
def gpu_mergesort(source, dest, size, width, slices, numThreads, numBlocks):
  idx = 0 # TODO replace with proper ID of current thread (scheme from C++ ago)
  # int x;
  # idx = threadIdx.x +
  #         threadIdx.y * (x  = threads->x) +
  #         threadIdx.z * (x *= threads->y) +
  #         blockIdx.x  * (x *= threads->z) +
  #         blockIdx.y  * (x *= blocks->z) +
  #         blockIdx.z  * (x *= blocks->y);
  
  start = width * idx * slices

  for s in slices:
    if start >= size:
      break
    
    middle = min(start + (width / 2), size)
    end = min(start + width, size)
    gpu_bottomUpMerge(source, dest, start, middle, end)
    start += width

@cuda.jit
# perform full MergeSort on supplied subsection of data
def gpu_bottomUpMerge(source, dest, start, middle, end):
  i = start
  j = middle

  for k in range(start, end):
    if i < middle and (j >= end || source[i] < source[j]):
      dest[j] = source[i]
      i += 1
    else:
      dest[k] = source[j]
      j += 1

# number of threads and blocks used on the GPU (up to (1024, 1024, 1024))
threadsPerBlock = (256, 1, 1)
blocksPerGrid = (512, 1, 1)
totalThreads = threadsPerBlock[0] * threadsPerBlock[1] * threadsPerBlock[2]
                * blocksPerGrid[0] * blocksPerGrid[1] * blocksPerGrid[2]

# create data array to be sorted
size = 100000
data = random.sample(range(size), size)
resultData = np.empty((data.shape))

# give pieces of the list to each thread
width = 2   # start at smallest reasonable split size (2 elements)
start = time.time()
while (width < (size * 2)):
  slices = size / ((totalThreads) * width) + 1
  gpu_mergesort[blocksPerGrid, threadsPerBlock](data, resultData, size, width, slices, threadsPerBlock, blocksPerGrid)


  # tempData = data 
  data = resultData
   
  # TODO do we need to "switch the input/output arrays instead of copying them around?"
    # data = data == D_data ? D_swp : D_data;
    # resultData = resultData == D_data ? D_swp : D_data;
  
  

  width *= 2

executionTime += time.time() - start

# TODO print the execution time
# TODO verify that the array was sorted properly
# TODO compare execution time to recursive non-parallelized algorithm

Average execution time is 0.0037560555934906007 with 1 threads.
Average execution time is 0.003610519886016846 with 2 threads.
Average execution time is 0.003678725481033325 with 4 threads.
Average execution time is 0.0037741470336914063 with 8 threads.
Average execution time is 0.004046255826950073 with 16 threads.
Average execution time is 0.004486570835113526 with 32 threads.
Average execution time is 0.0044991190433502195 with 64 threads.
Average execution time is 0.00448096489906311 with 128 threads.
Average execution time is 0.004564995765686035 with 256 threads.
Average execution time is 0.005304852247238159 with 512 threads.
Average execution time is 0.00730977201461792 with 1024 threads.
